In [1]:
import pandas as pd

- Para esse exemplo irei filtrar as mensagens bulk pq são mensagens de campanha eu quero saber a questão de qual o melhor canal para esse tipo de mensagem

In [2]:
chunks = pd.read_csv('../dataset/messages-demo.csv', chunksize=10_000_000)
df_msg = pd.concat(chunk[chunk["message_type"] == 'bulk'] for chunk in chunks)
df_msg.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_12552\1163188903.py:2: DtypeWarning: Columns (7,8,16,17,19,21,23,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_msg = pd.concat(chunk[chunk["message_type"] == 'bulk'] for chunk in chunks)


,id,message_id,campaign_id,message_type,client_id,channel,category,platform,email_provider,stream,...,is_soft_bounced,soft_bounced_at,is_complained,complained_at,is_blocked,blocked_at,is_purchased,purchased_at,created_at,updated_at
13107,54611899,1515915625624123395-608bc38f281ae,64,bulk,1515915625624123395,mobile_push,NaN,NaN,NaN,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-29 02:33:57.746309,2023-04-29 02:33:57.74715
13108,54616884,1515915625624529552-608bc39b67326,64,bulk,1515915625624529552,mobile_push,NaN,NaN,NaN,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-29 02:34:05.66109,2023-04-29 02:34:05.661892
13110,54852325,1515915625572420814-608bbb835b1f1,64,bulk,1515915625572420814,mobile_push,NaN,NaN,NaN,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-29 02:40:15.17397,2023-04-29 02:40:15.174785
13113,54998014,1515915625633534320-608bc6a4de53a,64,bulk,1515915625633534320,mobile_push,NaN,NaN,NaN,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-29 02:44:16.005064,2023-04-29 02:44:16.005879
73654,53967902,1515915625468066127-608bb03e9e608,63,bulk,1515915625468066127,mobile_push,NaN,NaN,NaN,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-29 02:17:12.013656,2023-04-29 02:17:12.014466


In [7]:
df_msg["date"] = pd.to_datetime(df_msg["date"], errors="coerce")
df_msg["sent_at"] = pd.to_datetime(df_msg["sent_at"], errors="coerce")
df_msg["clicked_first_time_at"] = pd.to_datetime(df_msg["clicked_first_time_at"], errors="coerce")

In [12]:
cols = ['id', 'message_id', 'campaign_id', 'message_type', 'client_id', 'channel', 'date', 'sent_at', 'is_clicked', 'clicked_first_time_at']
df_msg = df_msg[cols].head()

# Create Features

In [ ]:
## Criar hora, período e dia da semana

In [13]:
df_msg["hora_envio"] = df_msg["sent_at"].dt.hour
df_msg["periodo_dia"] = pd.cut(df_msg["hora_envio"],
                           bins=[-1, 5, 11, 17, 23],
                           labels=["madrugada", "manha", "tarde", "noite"])
df_msg["dia_semana"] = df_msg["sent_at"].dt.day_name()

In [14]:
df_msg.head()

,id,message_id,campaign_id,message_type,client_id,channel,date,sent_at,is_clicked,clicked_first_time_at,hora_envio,periodo_dia,dia_semana
13107,54611899,1515915625624123395-608bc38f281ae,64,bulk,1515915625624123395,mobile_push,2021-04-30,2021-04-30 08:45:03,f,NaT,8,manha,Friday
13108,54616884,1515915625624529552-608bc39b67326,64,bulk,1515915625624529552,mobile_push,2021-04-30,2021-04-30 08:45:15,f,NaT,8,manha,Friday
13110,54852325,1515915625572420814-608bbb835b1f1,64,bulk,1515915625572420814,mobile_push,2021-04-30,2021-04-30 08:10:43,f,NaT,8,manha,Friday
13113,54998014,1515915625633534320-608bc6a4de53a,64,bulk,1515915625633534320,mobile_push,2021-04-30,2021-04-30 08:58:13,f,NaT,8,manha,Friday
73654,53967902,1515915625468066127-608bb03e9e608,63,bulk,1515915625468066127,mobile_push,2021-04-30,2021-04-30 07:22:38,f,NaT,7,manha,Friday


In [16]:
# Tempo do ultimo clique
df_msg = df_msg.sort_values(by=["client_id", "sent_at"])
df_msg["ultimo_click"] = df_msg.groupby("client_id")["clicked_first_time_at"].shift(1)
df_msg["recencia_ultimo_click"] = (df_msg["sent_at"] - df_msg["ultimo_click"]).dt.days

In [17]:
df_msg.head()

,id,message_id,campaign_id,message_type,client_id,channel,date,sent_at,is_clicked,clicked_first_time_at,hora_envio,periodo_dia,dia_semana,ultimo_click,recencia_ultimo_click
73654,53967902,1515915625468066127-608bb03e9e608,63,bulk,1515915625468066127,mobile_push,2021-04-30,2021-04-30 07:22:38,f,NaT,7,manha,Friday,NaT,NaN
13110,54852325,1515915625572420814-608bbb835b1f1,64,bulk,1515915625572420814,mobile_push,2021-04-30,2021-04-30 08:10:43,f,NaT,8,manha,Friday,NaT,NaN
13107,54611899,1515915625624123395-608bc38f281ae,64,bulk,1515915625624123395,mobile_push,2021-04-30,2021-04-30 08:45:03,f,NaT,8,manha,Friday,NaT,NaN
13108,54616884,1515915625624529552-608bc39b67326,64,bulk,1515915625624529552,mobile_push,2021-04-30,2021-04-30 08:45:15,f,NaT,8,manha,Friday,NaT,NaN
13113,54998014,1515915625633534320-608bc6a4de53a,64,bulk,1515915625633534320,mobile_push,2021-04-30,2021-04-30 08:58:13,f,NaT,8,manha,Friday,NaT,NaN


In [ ]:
# Segmento Comportamental (Frequencia de Clique)
clique_por_cliente = df.groupby("client_id")["clicked_first_time_at"].count()
def segmentar(freq):
    if freq == 0:
        return "silencioso"
    elif freq < 3:
        return "ocasional"
    else:
        return "engajador"

df["segmento_comportamental"] = df["client_id"].map(clique_por_cliente).fillna(0).apply(segmentar)

In [ ]:
# Dia da Semana

In [ ]:
import pandas as pd

# Carregar o dataset
df = pd.read_csv("msg_sample.csv", parse_dates=["data_envio", "data_click"])

# 1. Período do dia



# 2. Recência do último clique por cliente


# 3. Segmento comportamental simples por frequência de clique


# 4. Dia da semana


# 5. Tipo da mensagem já está na coluna categoria_msg (renomear por clareza)


# 6. Canal de envio já está na coluna 'canal' (mantém como feature)
# Nenhuma alteração necessária



# Exibir resultado final
features_final = df[[
    "cliente_id", "data_envio", "periodo_dia", "recencia_ultimo_click",
    "segmento_comportamental", "dia_semana", "tipo_mensagem",
    "canal", "media_clique_ultimos_7_dias"
]]

import ace_tools as tools; tools.display_dataframe_to_user(name="Features CMAB", dataframe=features_final)

In [ ]:
# ==============================================================

In [ ]:
import pandas as pd
import numpy as np
from contextual_bandit.policy import LinUCB
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# ============================
# 1. Carregar e preparar o dataset
# ============================
df = pd.read_csv("msg_sample.csv")

# Corrigir colunas de data




# Corrigir clique: 't' → 1, 'f' → 0
df["recompensa"] = df["is_clicked"].map({"t": 1, "f": 0})

# Criar a ação: canal + categoria
df["acao"] = df["channel"].astype(str) + "_" + df["category"].astype(str)

# Remover registros incompletos
df = df.dropna(subset=["acao", "recompensa", "periodo_dia", "dia_semana"])

# ============================
# 2. Criar variáveis de contexto
# ============================
# Selecionar colunas de entrada
colunas_contexto = ["periodo_dia", "dia_semana", "channel", "category"]
X_contexto = pd.get_dummies(df[colunas_contexto])

# Ações codificadas em números
le_arms = LabelEncoder()
df["arm_idx"] = le_arms.fit_transform(df["acao"])

# ============================
# 3. Simular CMAB com LinUCB
# ============================
n_arms = df["arm_idx"].nunique()
n_features = X_contexto.shape[1]

model = LinUCB(n_arms=n_arms, n_features=n_features, alpha=0.1)

reward_history = []
cumulative_reward = 0

# Simulação interativa
for i in range(len(X_contexto)):
    contexto = X_contexto.iloc[i].values
    true_arm = df["arm_idx"].iloc[i]
    reward = df["recompensa"].iloc[i]

    # Predição do modelo
    predicted_arm = model.predict(contexto.reshape(1, -1))[0]

    # Feedback parcial
    observed_reward = reward if predicted_arm == true_arm else 0

    # Treino
    model.fit(contexto.reshape(1, -1), [predicted_arm], [observed_reward])

    cumulative_reward += observed_reward
    reward_history.append(cumulative_reward)

# ============================
# 4. Visualizar resultado
# ============================
plt.figure(figsize=(10, 5))
plt.plot(reward_history, label="CMAB - LinUCB")
plt.xlabel("Interações")
plt.ylabel("Recompensa acumulada (cliques)")
plt.title("Simulação de Contextual Multi-Armed Bandit (LinUCB)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()